In [1]:
import pandas as pd
import numpy as np
import random
import baselines as b
import re
from tqdm import tqdm
from sklearn.metrics import accuracy_score,f1_score, recall_score, precision_score, average_precision_score, balanced_accuracy_score

In [2]:



###===================


#path='./data/SAT_Predictions1/'
# path='./data/MRA_Files/'
path='./data/GPT2_Predictions/'
#path='./data/SAT_findings/'


def reports_cleaning(reports):
    
    new_reports=[]
    
    for r in list(reports):
        #print(r)
        new_reports.append(re.sub("\n|\r", " ", r))
    
    
    return new_reports
        


In [3]:
sample=[1,2,3,4,5]

bleu1,bleu2,bleu3,bleu4,M, R, C=[],[],[],[],[],[],[]

accuracy,precision,recall_mac,recall_mic=[],[],[],[]

bleu1_base,bleu2_base,bleu3_base,bleu4_base,M_base, R_base, C_base=[],[],[],[],[],[],[]

accuracy_base,precision_base,recall_mac_base,recall_mic_base=[],[],[],[]

classes=['No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly',
       'Lung Lesion', 'Lung Opacity', 'Edema', 'Consolidation', 'Pneumonia',
       'Atelectasis', 'Pneumothorax', 'Pleural Effusion', 'Pleural Other',
       'Fracture', 'Support Devices']

#### GPT2 read
test_orig=pd.read_csv(path+'gpt2_pred_labeled_'+str(0)+'.csv')
gt_orig=pd.read_csv(path+'gpt2_ref_labeled_'+str(0)+'.csv')

# test_orig['Reports']=reports_cleaning(test_orig['Reports'])
# gt_orig['Reports']=reports_cleaning(gt_orig['Reports'])

###### MRA reade
#gt=pd.read_csv(path+'gts_labelled'+str(1)+'.csv')
    #test=pd.read_csv(path+'res_labelled'+str(1)+'.csv')
    
##### SAT read

# test_orig=pd.read_csv(path+'score_labelled'+str(1)+'.csv')
# gt_orig=pd.read_csv(path+'score_ref_labelled'+str(1)+'.csv')


def missing_values_handeling(data):
    
    data=data.fillna(0.0)


    data=data.replace(-1,0)
    
    return data
    
def get_baseline(gt_shape):
    #baselines=pd.read_csv('./data/baselines_latest.csv')
    baselines=pd.read_csv('./data/baselines_with_cider.csv',encoding='latin-1')
    
    #print(baselines.iloc[:,0])
    base=baselines.iloc[3,:]

    base=pd.DataFrame(base).T

    base=pd.concat([base]*gt_shape, ignore_index=True)

    return base

def compute(i):
    
    rt_scores={}
    print('sample:', i)
    test=test_orig.copy()
    gt=gt_orig.copy()
    base_test=get_baseline(gt.shape[0])
    
    
    
    lst=range(480)
    indx=random.choices(lst,k=480)
    
    
    gt=gt.iloc[indx,:].reset_index(drop=True)

    test=test.iloc[indx,:].reset_index(drop=True)
    
    base_test=base_test.iloc[indx,:].reset_index(drop=True)
    
    
    
    
    #bese_test=test.sample(frac=1).reset_index(drop=True)
    
    
    gt=missing_values_handeling(gt)

    test=missing_values_handeling(test)
    base_test=missing_values_handeling(base_test)
    
    
    
    data=gt.iloc[:,1:]
    
    test_data=test.iloc[:,1:]
    
    base_data=base_test.iloc[:,1:]

    #test_data=test_data.sample(frac=1)
#     n=test_data.shape[0]
#     r=pd.DataFrame(data.iloc[report_id,:]).T

    pred = test_data#r * 1.0
    
    
    #### accuracy=======================================

    accuracy=accuracy_score(data.to_numpy().flatten(),test_data.to_numpy().flatten())
    
    
    #### accuracy baseline =======================================
    
    accuracy_base=accuracy_score(data.to_numpy().flatten(),base_data.to_numpy().flatten())

    #print(accuracy)
    
  #### Bleu scores
   
    
    #print(gt.shape,test.shape)

    bleus,meteor, rouge, ciders=b.all_scores((gt.iloc[:,0]),(test.iloc[:,0]))
    
    rt_scores['b']=bleus
    rt_scores['m']=meteor
    rt_scores['r']=rouge
    rt_scores['c']=ciders
    rt_scores['acc']=accuracy
#     #bleus,meteor, rouge, ciders=b.all_scores(np.array(gt.iloc[:,0])[is_normal], test.iloc[2,0])
    
#     bleu1.append(bleus[0])
#     bleu2.append(bleus[1])
#     bleu3.append(bleus[2])
#     bleu4.append(bleus[3])
#     M.append(meteor)
#     R.append(rouge)
#     C.append(ciders)
    
    #print(gt.shape,base_test.shape)

    bleus,meteor, rouge, ciders=b.all_scores((gt.iloc[:,0]),(base_test.iloc[:,0]))
    
#     #bleus,meteor, rouge, ciders=b.all_scores(np.array(gt.iloc[:,0])[is_normal], test.iloc[2,0])
    
#     bleu1_base.append(bleus[0])
#     bleu2_base.append(bleus[1])
#     bleu3_base.append(bleus[2])
#     bleu4_base.append(bleus[3])
#     M_base.append(meteor)
#     R_base.append(rouge)
#     C_base.append(ciders)

    rt_scores['b_base']=bleus
    rt_scores['m_base']=meteor
    rt_scores['r_base']=rouge
    rt_scores['c_base']=ciders
    rt_scores['acc_base']=accuracy_base
    
    return rt_scores
    

# for i in tqdm(range(0,1000)):
    
    
#     print('sample:', i)
#     compute()



In [4]:
### multiprocessing
import time
import multiprocessing as mp

# Step 1: Init multiprocessing.Pool()

pool = mp.Pool(mp.cpu_count())


t=time.time()

scores=pool.map(compute, [i for i in range(0,1000)])


print(time.time()-t)

sample: 126
sample: 0
sample: 63
sample: 189
{'testlen': 15314, 'reflen': 18038, 'guess': [15314, 14834, 14354, 13874], 'correct': [6545, 2552, 1138, 524]}
ratio: 0.848985475108058
{'testlen': 15544, 'reflen': 17954, 'guess': [15544, 15064, 14584, 14104], 'correct': [6555, 2403, 1011, 425]}
ratio: 0.8657680739667558
{'testlen': 15482, 'reflen': 18434, 'guess': [15482, 15002, 14522, 14042], 'correct': [6654, 2476, 1080, 472]}
{'testlen': 15362, 'reflen': 18016, 'guess': [15362, 14882, 14402, 13922], 'correct': [6702, 2593, 1132, 498]}
ratio: 0.8398611261798394
ratio: 0.8526865008880521
{'testlen': 16320, 'reflen': 17954, 'guess': [16320, 15840, 15360, 14880], 'correct': [7548, 2768, 1216, 411]}
ratio: 0.9089896401915502
{'testlen': 16320, 'reflen': 18038, 'guess': [16320, 15840, 15360, 14880], 'correct': [7683, 2916, 1375, 528]}
ratio: 0.9047566249029324
{'testlen': 16320, 'reflen': 18434, 'guess': [16320, 15840, 15360, 14880], 'correct': [7686, 3030, 1501, 667]}
ratio: 0.88532060323310

ratio: 0.8911090636876142
sample: 133
{'testlen': 15524, 'reflen': 18467, 'guess': [15524, 15044, 14564, 14084], 'correct': [6556, 2399, 975, 382]}
ratio: 0.8406346455839692
sample: 70
{'testlen': 15382, 'reflen': 18883, 'guess': [15382, 14902, 14422, 13942], 'correct': [6787, 2607, 1178, 562]}
ratio: 0.8145951384843079
{'testlen': 16320, 'reflen': 18992, 'guess': [16320, 15840, 15360, 14880], 'correct': [7815, 2954, 1416, 621]}
ratio: 0.8593091828137711
{'testlen': 16320, 'reflen': 17366, 'guess': [16320, 15840, 15360, 14880], 'correct': [7715, 3019, 1472, 659]}
ratio: 0.9397673615109443
{'testlen': 16320, 'reflen': 18467, 'guess': [16320, 15840, 15360, 14880], 'correct': [7558, 2858, 1387, 624]}
ratio: 0.8837385606757522
{'testlen': 16320, 'reflen': 18883, 'guess': [16320, 15840, 15360, 14880], 'correct': [7746, 2962, 1401, 579]}
ratio: 0.8642694487104345
sample: 197
sample: 8
{'testlen': 15425, 'reflen': 18768, 'guess': [15425, 14945, 14465, 13985], 'correct': [6804, 2646, 1139, 464

ratio: 0.8212976934683757
{'testlen': 16320, 'reflen': 18340, 'guess': [16320, 15840, 15360, 14880], 'correct': [7866, 3004, 1487, 646]}
ratio: 0.8898582333696352
{'testlen': 16320, 'reflen': 17893, 'guess': [16320, 15840, 15360, 14880], 'correct': [7766, 3112, 1591, 759]}
ratio: 0.9120885262392605
{'testlen': 16320, 'reflen': 18891, 'guess': [16320, 15840, 15360, 14880], 'correct': [7766, 2873, 1300, 490]}
ratio: 0.8639034460853917
{'testlen': 16320, 'reflen': 18556, 'guess': [16320, 15840, 15360, 14880], 'correct': [7843, 2973, 1397, 567]}
ratio: 0.8794998922181031
sample: 204
{'testlen': 15688, 'reflen': 18148, 'guess': [15688, 15208, 14728, 14248], 'correct': [6658, 2606, 1200, 593]}
ratio: 0.8644478730438139
sample: 15
{'testlen': 15450, 'reflen': 17912, 'guess': [15450, 14970, 14490, 14010], 'correct': [6656, 2538, 1123, 500]}
ratio: 0.8625502456453292
sample: 78
sample: 141
{'testlen': 15469, 'reflen': 18324, 'guess': [15469, 14989, 14509, 14029], 'correct': [6676, 2507, 1055, 4

ratio: 0.9222944334557264
{'testlen': 16320, 'reflen': 18041, 'guess': [16320, 15840, 15360, 14880], 'correct': [7817, 3046, 1473, 605]}
ratio: 0.9046061748239618
{'testlen': 16320, 'reflen': 17990, 'guess': [16320, 15840, 15360, 14880], 'correct': [7696, 3073, 1539, 689]}
ratio: 0.9071706503612614
sample: 22
sample: 211
{'testlen': 15189, 'reflen': 17734, 'guess': [15189, 14709, 14229, 13749], 'correct': [6629, 2603, 1215, 602]}
ratio: 0.8564903575053087
{'testlen': 15398, 'reflen': 18516, 'guess': [15398, 14918, 14438, 13958], 'correct': [6838, 2620, 1169, 533]}
ratio: 0.831605098293323
sample: 85
{'testlen': 15149, 'reflen': 18147, 'guess': [15149, 14669, 14189, 13709], 'correct': [6602, 2512, 1053, 428]}
ratio: 0.834793629801023
sample: 148
{'testlen': 15160, 'reflen': 18255, 'guess': [15160, 14680, 14200, 13720], 'correct': [6686, 2659, 1216, 582]}
ratio: 0.830457408929015
{'testlen': 16320, 'reflen': 17734, 'guess': [16320, 15840, 15360, 14880], 'correct': [7684, 2953, 1378, 523]

ratio: 0.8668862211834236
sample: 29
{'testlen': 15487, 'reflen': 18012, 'guess': [15487, 15007, 14527, 14047], 'correct': [6457, 2397, 1061, 496]}
ratio: 0.85981567843655
sample: 218
sample: 92
{'testlen': 15241, 'reflen': 18200, 'guess': [15241, 14761, 14281, 13801], 'correct': [6636, 2575, 1178, 553]}
ratio: 0.8374175824175364
{'testlen': 15410, 'reflen': 17906, 'guess': [15410, 14930, 14450, 13970], 'correct': [6565, 2460, 1084, 489]}
ratio: 0.86060538367023
sample: 155
{'testlen': 15531, 'reflen': 18375, 'guess': [15531, 15051, 14571, 14091], 'correct': [6829, 2629, 1177, 557]}
ratio: 0.8452244897958724
{'testlen': 16320, 'reflen': 18012, 'guess': [16320, 15840, 15360, 14880], 'correct': [7660, 2922, 1426, 627]}
ratio: 0.9060626249166719
{'testlen': 16320, 'reflen': 18200, 'guess': [16320, 15840, 15360, 14880], 'correct': [7814, 3015, 1438, 618]}
ratio: 0.8967032967032474
{'testlen': 16320, 'reflen': 17906, 'guess': [16320, 15840, 15360, 14880], 'correct': [7657, 2839, 1294, 474]}

...

ratio: 0.8504631427144683
sample: 973
{'testlen': 15348, 'reflen': 18100, 'guess': [15348, 14868, 14388, 13908], 'correct': [6511, 2489, 1155, 541]}
ratio: 0.8479558011049255
{'testlen': 16320, 'reflen': 17998, 'guess': [16320, 15840, 15360, 14880], 'correct': [7665, 3005, 1491, 669]}
ratio: 0.9067674186020165
{'testlen': 16320, 'reflen': 18167, 'guess': [16320, 15840, 15360, 14880], 'correct': [7647, 2980, 1449, 607]}
ratio: 0.8983321406946166
{'testlen': 16320, 'reflen': 18029, 'guess': [16320, 15840, 15360, 14880], 'correct': [7673, 2891, 1370, 553]}
ratio: 0.9052082755559984
{'testlen': 16320, 'reflen': 18100, 'guess': [16320, 15840, 15360, 14880], 'correct': [7568, 2872, 1369, 509]}
ratio: 0.9016574585634861
sample: 785
{'testlen': 15753, 'reflen': 18251, 'guess': [15753, 15273, 14793, 14313], 'correct': [6798, 2494, 1086, 476]}
ratio: 0.8631307873540702
sample: 848
{'testlen': 15439, 'reflen': 18435, 'guess': [15439, 14959, 14479, 13999], 'correct': [6648, 2519, 1095, 465]}
ratio

ratio: 0.9281164695176906
{'testlen': 16320, 'reflen': 18931, 'guess': [16320, 15840, 15360, 14880], 'correct': [7783, 3024, 1476, 635]}
ratio: 0.8620780730019089
{'testlen': 16320, 'reflen': 17793, 'guess': [16320, 15840, 15360, 14880], 'correct': [7900, 3212, 1624, 756]}
ratio: 0.9172146349687564
{'testlen': 16320, 'reflen': 18386, 'guess': [16320, 15840, 15360, 14880], 'correct': [7830, 3013, 1489, 660]}
ratio: 0.8876318938322154
sample: 792
{'testlen': 15435, 'reflen': 18031, 'guess': [15435, 14955, 14475, 13995], 'correct': [6556, 2414, 1025, 461]}
ratio: 0.8560257334589952
sample: 855
{'testlen': 15362, 'reflen': 18167, 'guess': [15362, 14882, 14402, 13922], 'correct': [6636, 2478, 1095, 499]}
sample: 918
ratio: 0.8455991633180577
{'testlen': 15454, 'reflen': 18628, 'guess': [15454, 14974, 14494, 14014], 'correct': [6758, 2626, 1178, 530]}
ratio: 0.8296113377710527
sample: 981
{'testlen': 15485, 'reflen': 17643, 'guess': [15485, 15005, 14525, 14045], 'correct': [6712, 2583, 1149,

ratio: 0.8703999999999535
{'testlen': 16320, 'reflen': 18129, 'guess': [16320, 15840, 15360, 14880], 'correct': [7782, 3086, 1497, 673]}
ratio: 0.900215124937895
sample: 799
{'testlen': 15452, 'reflen': 18364, 'guess': [15452, 14972, 14492, 14012], 'correct': [6650, 2438, 1047, 453]}
ratio: 0.8414288825963384
sample: 862
{'testlen': 15469, 'reflen': 18431, 'guess': [15469, 14989, 14509, 14029], 'correct': [6656, 2536, 1106, 485]}
ratio: 0.8392924963376464
sample: 925
{'testlen': 15532, 'reflen': 18139, 'guess': [15532, 15052, 14572, 14092], 'correct': [6710, 2574, 1143, 528]}
ratio: 0.8562765312310019
sample: 988
{'testlen': 15473, 'reflen': 18127, 'guess': [15473, 14993, 14513, 14033], 'correct': [6931, 2841, 1356, 710]}
ratio: 0.8535885695371074
{'testlen': 16320, 'reflen': 18364, 'guess': [16320, 15840, 15360, 14880], 'correct': [7855, 2948, 1376, 528]}
ratio: 0.8886952733608752
{'testlen': 16320, 'reflen': 18431, 'guess': [16320, 15840, 15360, 14880], 'correct': [7788, 3053, 1486, 

ratio: 0.8443286094368874
sample: 869
{'testlen': 15392, 'reflen': 18268, 'guess': [15392, 14912, 14432, 13952], 'correct': [6721, 2637, 1207, 594]}
ratio: 0.8425662360411187
sample: 932
{'testlen': 15550, 'reflen': 18042, 'guess': [15550, 15070, 14590, 14110], 'correct': [6704, 2545, 1080, 473]}
ratio: 0.8618778405941213
sample: 995
{'testlen': 15284, 'reflen': 18308, 'guess': [15284, 14804, 14324, 13844], 'correct': [6701, 2571, 1119, 493]}
ratio: 0.8348263054401991
{'testlen': 16320, 'reflen': 18417, 'guess': [16320, 15840, 15360, 14880], 'correct': [7809, 3135, 1542, 648]}
ratio: 0.8861378074604503
{'testlen': 16320, 'reflen': 18268, 'guess': [16320, 15840, 15360, 14880], 'correct': [7853, 3192, 1631, 780]}
ratio: 0.8933654477774856
{'testlen': 16320, 'reflen': 18042, 'guess': [16320, 15840, 15360, 14880], 'correct': [7717, 2974, 1377, 532]}
ratio: 0.9045560359161454
{'testlen': 16320, 'reflen': 18308, 'guess': [16320, 15840, 15360, 14880], 'correct': [7757, 2972, 1388, 526]}
ratio

ratio: 0.8587123709074457
sample: 877
{'testlen': 15310, 'reflen': 17790, 'guess': [15310, 14830, 14350, 13870], 'correct': [6751, 2698, 1254, 588]}
ratio: 0.8605958403597043
sample: 940
{'testlen': 15502, 'reflen': 18506, 'guess': [15502, 15022, 14542, 14062], 'correct': [6605, 2495, 1073, 442]}
ratio: 0.8376742678049909
{'testlen': 16320, 'reflen': 18204, 'guess': [16320, 15840, 15360, 14880], 'correct': [7829, 3073, 1535, 705]}
ratio: 0.8965062623598716
{'testlen': 16320, 'reflen': 17790, 'guess': [16320, 15840, 15360, 14880], 'correct': [7952, 3190, 1558, 699]}
ratio: 0.9173693086002856
{'testlen': 16320, 'reflen': 18506, 'guess': [16320, 15840, 15360, 14880], 'correct': [7826, 3046, 1483, 637]}
ratio: 0.8818761482761871
sample: 815
{'testlen': 15341, 'reflen': 18262, 'guess': [15341, 14861, 14381, 13901], 'correct': [6851, 2660, 1196, 549]}
ratio: 0.8400503778337072
sample: 878
{'testlen': 15310, 'reflen': 18109, 'guess': [15310, 14830, 14350, 13870], 'correct': [6602, 2544, 1115,

In [5]:

for sample in scores:
    
    bleu1.append(sample['b'][0])
    bleu2.append(sample['b'][1])
    bleu3.append(sample['b'][2])
    bleu4.append(sample['b'][3])
    M.append(sample['m'])
    R.append(sample['r'])
    C.append(sample['c'])
    accuracy.append(sample['acc'])
    
    bleu1_base.append(sample['b_base'][0])
    bleu2_base.append(sample['b_base'][1])
    bleu3_base.append(sample['b_base'][2])
    bleu4_base.append(sample['b_base'][3])
    M_base.append(sample['m_base'])
    R_base.append(sample['r_base'])
    C_base.append(sample['c_base'])
    accuracy_base.append(sample['acc_base'])
    


    
data=pd.DataFrame([])

data['b1']=bleu1
data['b2']=bleu2
data['b3']=bleu3
data['b4']=bleu4
data['M']=M
data['R']=R
data['C']=C
data['accuracy']=accuracy

data['b1_base']=bleu1_base
data['b2_base']=bleu2_base
data['b3_base']=bleu3_base
data['b4_base']=bleu4_base
data['M_base']=M_base
data['R_base']=R_base
data['C_base']=C_base
data['accuracy_base']=accuracy_base

#print(np.mean(accuracy))
data.to_csv('samples_GPT2_baseCider0.csv')